<a href="https://colab.research.google.com/github/Felipe-Souza-Marra/GameIA_Fingers/blob/main/GameToStartIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ten fingers

# Objetivo

* Consiste em zerar os dedos das duas mões do adversário.

# Regras

* O jogo começa com dois jogadores que suas mões com apenas um dedo inicialmente.
* No máximo 5 jogadas específicas que contém sua condições também:
 * 4 jogadas vão consistir em passar a quantidade de dedos que você possui em uma mão para outra do adversário sem diminuir a sua.
 * Se uma mão estiver zerada ela não poderá ser usada, ficando com apenas 2 jogadas.
 * E a última jogada vai consistir em dividir um valor par do dedos de uma mão para sua outra mão que precisa estar vazia(Com 0 dedos).




In [4]:
# From
from enum import Enum

# Import
import numpy as np
import os

In [5]:
class HandType(Enum):
  RIGHT = "right"
  LEFT = "left" 

class ActionType(Enum):
  PLACE = "place"
  SPLIT = "split"

class StatusHands(Enum):
  INGAME = "inGame"
  LOST = "lost"

In [6]:
class Hand(object):
  def __init__(self, handType: HandType, fingers: int = 1):
    self._fingers = fingers
    self.__HandType = handType

  def _checkFingers(self):
    if self._fingers >= 5:
      self._fingers -= 5

  def _canUseHand(self):
    if self._fingers == 0:
      return False
    return True

testHand = Hand(HandType.LEFT)

In [7]:
class Action(object):
  def __init__(self, action: ActionType, fromHand: Hand, toHand: Hand):
    self.__actionType = action
    self.__fromHand = fromHand
    self.__toHand = toHand

  def _realize(self):
    if self.__actionType == ActionType.PLACE:
      self.__toHand._fingers += self.__fromHand._fingers
      self.__toHand._checkFingers()
    else:
      self.__toHand._fingers = int(self.__fromHand._fingers / 2)
      self.__fromHand._fingers = int(self.__fromHand._fingers / 2)

In [8]:
class Player(object):
  def __init__(self):
    self._leftHand = Hand(HandType.LEFT)
    self._rightHand = Hand(HandType.RIGHT)

  def _canSplit(self):
    if self._leftHand._fingers > 0 and self._rightHand._fingers > 0:
      return False
    elif self._leftHand._fingers != 0 and self._leftHand._fingers % 2 != 0:
      return False
    elif self._rightHand._fingers != 0 and self._rightHand._fingers % 2 != 0:
      return False
    return True

  def _statusFromHands(self):
    if self._leftHand._fingers > 0 or self._rightHand._fingers > 0:
      return StatusHands.INGAME
    return StatusHands.LOST

  def _putOnHand(self, action: Action):
    action._realize()

testPlayer = Player()

In [11]:
class Game(object):
  def __init__(self, playerOne: Player, playerTwo: Player):
    self._playerOne = playerOne
    self._playerTwo = playerTwo
    self.__maxRounds = 20
    self.__gameFinished = False
    self.__round = 1

  def _finishGame(self):
    if self._playerOne._statusFromHands() == StatusHands.LOST:
      self.__gameFinished = True
    elif self._playerTwo._statusFromHands() == StatusHands.LOST:
      self.__gameFinished = True
    elif self.__maxRounds == self.__round:
      self.__gameFinished = True
  
  def returnWinner(self):
    if self._playerOne._statusFromHands() == StatusHands.LOST:
      return self._playerTwo
    elif self._playerTwo._statusFromHands() == StatusHands.LOST:
      return self._playerOne
    return None

  def _showHands(self, player: Player, challenger: Player):
    print("\n" + str(challenger._rightHand._fingers), end="") 
    print("<------------>" + str(challenger._leftHand._fingers),)
    print("\n" + str(player._leftHand._fingers), end="")
    print("<------------>" + str(player._rightHand._fingers), end="\n\n")

  def startGame(self):
    while not self.__gameFinished:
      
      self._possibleMoves(self._playerOne, self._playerTwo)

      self._possibleMoves(self._playerTwo, self._playerOne)

      # print(self.__round)
      self.__round += 1

  def _possibleMoves(self,player: Player, challenger: Player):
    self._finishGame()
    while True and not(self.__gameFinished):
      possibilities = []

      # Uso da mão esquerda
      if player._leftHand._canUseHand():
        if challenger._leftHand._canUseHand():
          possibilities.append(("Mão esquerda a mão esquerda",
                                  Action(ActionType.PLACE, player._leftHand,
                                          challenger._leftHand)))
          
        if challenger._rightHand._canUseHand():
          possibilities.append(("Mão esquerda a mão direita",
                                  Action(ActionType.PLACE, player._leftHand,
                                          challenger._rightHand)))
      
      # Uso da mão direita
      if player._rightHand._canUseHand():
        if challenger._leftHand._canUseHand():
          possibilities.append(("Mão direita a mão esquerda",
                                  Action(ActionType.PLACE, player._rightHand,
                                          challenger._leftHand)))
          
        if challenger._rightHand._canUseHand():
          possibilities.append(("Mão direita a mão direita",
                                  Action(ActionType.PLACE, player._rightHand,
                                          challenger._rightHand)))
          
      # Dividir o valor entre as mãos
      canSplit = player._canSplit()
      if canSplit and player._leftHand._canUseHand():
        possibilities.append(("Dividir",
                                  Action(ActionType.SPLIT, player._leftHand,
                                          player._rightHand)))
      elif canSplit and player._rightHand._canUseHand():
        possibilities.append(("Dividir",
                                  Action(ActionType.SPLIT, player._rightHand,
                                          player._leftHand)))
      while True:
        self._showHands(player, challenger)
        for i in range(len(possibilities)):
          print(str(i + 1) + " - " + possibilities[i][0])

        choose = int(input("Escolha um: "))

        if choose > 0 and choose < len(possibilities) + 1:
          break
      
      player._putOnHand(possibilities[choose - 1][1])

      break

testGame = Game(Player(), Player())

testGame.startGame()

print(testGame.returnWinner())

# Alphabeta prunum
# Poda alfa beta
# SBGames


1<------------>1

1<------------>1

1 - Mão esquerda a mão esquerda
2 - Mão esquerda a mão direita
3 - Mão direita a mão esquerda
4 - Mão direita a mão direita
Escolha um: 1
Escolha um: 1

1<------------>1

2<------------>1

1 - Mão esquerda a mão esquerda
2 - Mão esquerda a mão direita
3 - Mão direita a mão esquerda
4 - Mão direita a mão direita

1<------------>2

3<------------>1

1 - Mão esquerda a mão esquerda
2 - Mão esquerda a mão direita
3 - Mão direita a mão esquerda
4 - Mão direita a mão direita


KeyboardInterrupt: ignored